# packages 

In [61]:
import numpy as np

In [63]:
def get_security_transition_matrix():
    # The transition matrix is the same regardless if the game is cycled or not
    # /!\ index different from square number (need to do -1)
    n_squares = 15 
    T_m = np.zeros((n_squares, n_squares))
    #i = index from ==> on ne commence jamais à cet index car on gagné à ce moment. La proba =1. C'est mis après 
    for i in range(n_squares-1): 
        T_m[i][i] = 1/2
        if(i==2):
            T_m[i][i+1] = 1/4 #if takes slow line
            T_m[i][10] = 1/4 #if takes fast line
        elif(i==9):
            T_m[i][14] = 1/2            
        else:
            T_m[i][i+1] = 1/2
    T_m[14][14] = 1
    return T_m

def get_normal_transition_matrix(layout, cycle:bool):
    # The transition matrix is the same regardless tif the game is cycled or not
    # If the player just passes through square 3 without stopping, 
    # he continues to square 4 or beyond (5, 6, etc), as if the other path does not exist
    n_squares = 15 
    T_m = np.zeros((n_squares, n_squares))
    for i in range(n_squares-1): # o
        T_m[i][i] = 1/3
        if(i==2):
            # #if takes slow line
            T_m[i][i+1] = 1/6 
            T_m[i][i+2] = 1/6
            #if takes fast line
            T_m[i][10] = 1/6
            T_m[i][11] = 1/6

        elif(i==8):
            T_m[i][i+1] = 1/3
            T_m[i][14] = 1/3

        elif(i==9 or i==13):
            if(cycle): 
                T_m[i][14] = 1/3
                T_m[i][0] = 1/3
            else:
                T_m[i][14] = 2/3

        else:
            T_m[i][i+1] = 1/3
            T_m[i][i+2] = 1/3
    T_m[n_squares-1][n_squares-1] = 1

    
    # Il faut encore prendre les pièges et bonuses en compte 
    #il faut reparcourir Chaque élément de la T_m
    for i in range(n_squares-1): # on ne veut pas changer le point de départ ni le point d'arrivé car il n'y a ni piège 
        for j in range(n_squares-1):
                # une chance sur 2 de trigger le piège ou bonus
                if(layout[j]==1): # type 1: go back at 1st square
                    T_m[i][0] += T_m[i][j]/2 # il faut transférer une demi proba vers le square 1
                elif (layout[j]==2): #type 2: go back 3 squares
                    #il y a qlq cas spéciaux
                    if (j>=10 and j <=12): # if j ==11 12 or 13 --> go to 1,2 or 3 with proba 1/2
                        #diff = 10
                        T_m[i][j-10] += T_m[i][j]/2
                        
                    # si trap type 2 à la case 2 ou 3, on peut pas aller moins loin que le start 
                    elif(j<3):
                        T_m[i][0] += T_m[i][j]/2 # il faut transférer une demi proba vers le square 1
                        
                    else: # cas "commun"
                        T_m[i][j-3] += T_m[i][j]/2
                T_m[i][j] /=2 # une fois le transfert de proba fait, il faut bien l'enlever en [i][j] forcément
    return T_m

def get_risky_transition_matrix(layout, cycle:bool):
    
    n_squares = 15
    T_m = np.zeros((n_squares, n_squares))

    for i in range(n_squares-1): # on prend le dernier square après la boucle, d'où le -1

        T_m[i][i] = 1/4
        if(i==2):
            # #if takes slow line
            T_m[i][i+1] = 1/8 
            T_m[i][i+2] = 1/8
            T_m[i][i+3] = 1/8
            #if takes fast line
            T_m[i][10] = 1/8
            T_m[i][11] = 1/8
            T_m[i][12] = 1/8
        
        elif(i==7):
            T_m[i][i+1] = 1/4
            T_m[i][i+2] = 1/4
            T_m[i][14] = 1/4 #(du 10 au 15)

        elif(i==8 or i==12):
            T_m[i][9] = 1/4
            if cycle :
                T_m[i][14] = 1/4
                T_m[i][0] = 1/4
            else : 
                T_m[i][14] = 1/2
        
        elif(i==9 or i == 13):
            if cycle :
                T_m[i][14] = 1/4
                T_m[i][0] = 1/4
                T_m[i][1] = 1/4
            else :
                T_m[i][14] = 3/4
        
        else : 
            T_m[i][i+1] = 1/4
            T_m[i][i+2] = 1/4
            T_m[i][i+3] = 1/4
    
    T_m[n_squares-1][n_squares-1] = 1
    
    # LES PIEGES :
    for i in range(n_squares-1): # on ne veut pas changer le point de départ ni le point d'arrivé car il n'y a ni piège 
        for j in range(n_squares-1):
            # Le trap est d'office triggered si c'en est un
                if(layout[j]==1): # type 1: go back at 1st square
                    T_m[i][0] += T_m[i][j]
                elif(layout[j]== 2): # -3 squares
                    if (j>=10 and j <=12):
                         #diff = 10
                        T_m[i][j-10] += T_m[i][j]
                    # si trap type 2 à la case 2 ou 3, on peut pas aller moins loin que le start 
                    elif(j<3):
                        T_m[i][0] += T_m[i][j] # il faut transférer une demi proba vers le square 1
                    else: # cas "commun"
                        T_m[i][j-3] += T_m[i][j]
                T_m[i][j] = 0 # une fois le transfert de proba fait, il faut bien l'enlever en [i][j] forcément





    return T_m

In [128]:
class SimulateAGame():

    dice_list = []
    current_index_list = []
    index_from_list = []
    layout_index = []

    def __init__(self, layout, circle):
        self.layout =layout
        self.circle = circle
        self.cost = 0
    
    def get_cost_game(self, policy):
        """
        policy says which dice to take via int number: 
        - 1 : security
        - 2 : normal 
        - 3 : risky
        """
        self.cost = 0
        return self.get_cost_from(policy, 0)
    
    def get_cost_from(self, policy, index_from):
        """ 
        """
        #assert(len(policy) == len(self.layout))
        current_index = index_from 
        while(current_index != 14):
            dice = policy[current_index] #{1,2,3}
            if(dice not in [1,2,3]): raise ValueError('sorry zobi')
            # print([current_index, self.cost, index_from])
            current_index = self.play(current_index, dice)
            self.cost+=1    
            # Purpose of pointing out some error
            self.dice_list.append(dice)
            self.current_index_list.append(current_index)
            self.index_from_list.append(index_from)
            self.layout_index.append(self.layout[current_index])
        return self.cost

    def play(self, current_index, dice_type):
        """return resulting index"""        
        if( dice_type == 1):  # no traps nor bonuses
            if(current_index == 2): return np.random.choice([2,3,10], p = [0.5, 0.25, 0.25] )
            elif (current_index == 9): return np.random.choice([9,14] )
            else: return np.random.choice([current_index, current_index+1])
        elif(dice_type ==2): # normal dice: 1/2 to trigger trap/bonus
            is_triggered = np.random.choice([True,False]) #50% chance to trigger trap/bonus

            if(current_index==2): next_index = np.random.choice([current_index,3,4,10,11], p = [1/3, 1/6, 1/6, 1/6, 1/6])
            elif(current_index==8): next_index = np.random.choice([current_index,9,14])
            elif(current_index==9 or current_index == 13):
                if(self.circle): 
                    next_index = np.random.choice([current_index,14,0])
                else: next_index = np.random.choice([current_index,14], p = [1/3,2/3])

            # Ne faut il pas ajouté pour tous les autres cas ?
            else : next_index = np.random.choice([current_index, current_index+1, current_index+2])
                
            if (is_triggered and next_index!=14): return self.activateTrapOrBonus(next_index)
            else: return next_index

        else: # type =3, i.e. risky
            if(current_index==2):
                next_index = np.random.choice([current_index,3,4,5, 10,11,12], p = [1/4, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8])
            elif(current_index==7):
                next_index = np.random.choice([current_index,8,9,14])
            elif(current_index==8):
                if(self.circle): next_index = np.random.choice([current_index,9,14,0])
                else: next_index = np.random.choice([current_index,9,14], p = [1/4, 1/4, 2/4])
            elif(current_index==9):
                if(self.circle): next_index = np.random.choice([current_index,14,0,1])
                else: next_index = np.random.choice([current_index,14], p = [1/4, 3/4])
            elif(current_index==12):
                if(self.circle): next_index = np.random.choice([current_index,13,14,0])
                else: next_index = np.random.choice([current_index,13,14], p = [1/4, 1/4, 2/4])
            elif(current_index==13):
                if(self.circle): next_index = np.random.choice([current_index,14,0,1])
                else: next_index = np.random.choice([current_index,14], p = [1/4, 3/4])
            
            else: next_index = np.random.choice([current_index, current_index+1 , current_index+2, current_index+3])

            return self.activateTrapOrBonus(next_index)

    def activateTrapOrBonus(self,index):
        if (self.layout[index] == 1): return self.activateType1Trap(index)
        elif (self.layout[index] == 2): return self.activateType2Trap(index)
        elif (self.layout[index] == 3): return self.activateType3Trap(index)
        elif (self.layout[index] == 4): return self.activateBonus(index)
        else: return index # ordinary square (0)

    def activateType1Trap(self, index):
        if (index!=14):
            return 0
        else : return index


    def activateType2Trap(self, index):
        if (index!=14):
            if(index<3 or index == 10): 
                return 0 # or index = 10 je crois
            elif(index == 11 or index == 12): 
                return (index-10) # Entre 11 et 12 et non entre 10 et 12
            else: 
                return (index-3)
        else : return index

    def activateType3Trap(self, index):
        if (index!=14):
            self.cost += 1
        return index

    def activateBonus(self, index):
        if (index!=14):
            self.cost -= 1
        return index
    

    def get_game_log(self):
        for i in range(len(self.dice_list)):
            if i == 0:
                print('dice played : {0} leading from index [{1}] to index [{2}] that is a trap/bonus {3}'.format(self.dice_list[i]+1, 
                                                                                                                  1, 
                                                                                                                  self.current_index_list[i]+1, 
                                                                                                                  self.layout_index[i]))

            else:
                print('dice played : {0} leading from index [{1}] to index [{2}] that is a trap/bonus {3}'.format(self.dice_list[i]+1, 
                                                                                                                  self.current_index_list[i-1]+1, 
                                                                                                                  self.current_index_list[i]+1, 
                                                                                                                  self.layout_index[i]))
        print('with a total cost of {0}'.format(self.cost))



In [138]:
L_rand = [0,1,2,3,4,0,1,2,3,4,0,1,2,3,4]
L_no_trap = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
secu_policy = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
rand_policy = [1,2,3,1,2,3,1,2,3,1,2,3,1,2,3]
aGame = SimulateAGame(layout= L_no_trap, circle= False)
aGame.get_cost_game(policy= [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
aGame.get_game_log()


dice played : 4 leading from index [1] to index [1] that is a trap/bonus 0
dice played : 4 leading from index [1] to index [1] that is a trap/bonus 0
dice played : 4 leading from index [1] to index [1] that is a trap/bonus 0
dice played : 4 leading from index [1] to index [1] that is a trap/bonus 0
dice played : 4 leading from index [1] to index [1] that is a trap/bonus 0
dice played : 4 leading from index [1] to index [1] that is a trap/bonus 0
dice played : 4 leading from index [1] to index [4] that is a trap/bonus 3
dice played : 4 leading from index [4] to index [5] that is a trap/bonus 4
dice played : 3 leading from index [5] to index [6] that is a trap/bonus 0
dice played : 2 leading from index [6] to index [6] that is a trap/bonus 0
dice played : 2 leading from index [6] to index [6] that is a trap/bonus 0
dice played : 2 leading from index [6] to index [6] that is a trap/bonus 0
dice played : 2 leading from index [6] to index [6] that is a trap/bonus 0
dice played : 2 leading f

In [5]:
class markovDecision():
    def __init__(self, layout:  np.ndarray, cycle:bool):
        self.security_transition_matrix = get_security_transition_matrix()
        self.normal_transition_matrix = get_normal_transition_matrix(layout,cycle)
        self.get_risky_transition_matrix = get_risky_transition_matrix(layout,cycle)
    
    # at each step, one needs to choose which dice, which means which transitions matrix on which to play in order to minimise the expected number of tosses
    
    def run(self):
        """ 
        return Expec (length = 14) and Dice with the optimal dice choice
        """
        pass

In [6]:
class Board():
    # here we place class attribute (static members)
    # init 
    def __init__(self, layout:  np.ndarray, cycle=False):
        self.layout = layout
        self.cycle = cycle
        # self.squares = np.ones(15) # matrice 15 x 3 avec les actions et expected costs associés. 
        self.position = 0 #position actuelle du joueur
    
    # regular step
    def step(self, size):
        self.position += size

    # reset (in case of circular)
    def reset_position(self):
        self.position = 0

    def get_position(self):
        return self.position
    
    def print(self):
        pass

In [7]:
class Dice():
    def __init__(self, type:int):
        self.last_toss = None

        if(type == 1):
            self.steps = np.array([0,1])
        elif (type == 2):
            self.steps = np.array([0,1,2])
        elif (type== 3):
            self.steps = np.array([0,1,2,3])
        else: raise ValueError(' /!\  Dice type not existing in current settings')
        

    def roll(self):
        """ 
        draw uniformly a step from the dice
        """
        self.last_toss =  int(np.random.choice(self.steps, size=1))
        return self.last_toss
    
    def get_current_toss(self):
        if(self.last_toss==None): raise ValueError(' /!\ Dice should have been tossed at least once')
        return self.last_toss

In [8]:
def get_proba_next_move_position(v_k, T_m):
    # v_k = current_state proba vector
    # return next state proba vector
    return np.dot(v_k, T_m)

In [9]:
v_0 = np.zeros(16)
v_0[1] = 1
security_transition_matrix = get_security_transition_matrix()
np.dot(v_0, security_transition_matrix) # représente la proba de la prochaine position

IndexError: index 15 is out of bounds for axis 0 with size 15